In [69]:
import pandas as pd
from selenium import webdriver
import pandas as pd
import numpy as np
import pickle
import itertools

from collections import defaultdict


import datetime
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime

from library import Dataset
from Database import update_sentiment
from Crawler.settings import REPORT_TYPE_MAPPING
from uti import UrlManager, Logger, By, DataLoader

DL = DataLoader()
UM = UrlManager()
logger = Logger()

In [3]:

# warnings.filterwarnings("ignore")
# from xbbg import blp



In [317]:
from Database.report_type import rc_filter, er_filter, io_filter, ec_filter

In [4]:
driver = UM.start_browser()



2022-05-25 19:06:31,014 P[19996] [MainThread 12952] 

====== WebDriver manager ======
2022-05-25 19:06:31,015 P[19996] [MainThread 12952] ====== WebDriver manager ======
Current google-chrome version is 101.0.4951
2022-05-25 19:06:31,031 P[19996] [MainThread 12952] Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
2022-05-25 19:06:31,032 P[19996] [MainThread 12952] Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\JayYang\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
2022-05-25 19:06:31,096 P[19996] [MainThread 12952] Driver [C:\Users\JayYang\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


19:06:31 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\Log/20220525/20220525190434.txt created.
19:06:31 Starting Browser


In [24]:
UM.get_url(driver, 'https://nbf-library.bluematrix.com/client/library.jsp')

16:08:55: Accessing url... 16:08:58 Success. Idle for 5 seconds...


In [8]:
import pdb

In [22]:
def nbc_login(driver, need_un=True):
    username = 'donnert@a-s-capital.com'
    password ='Pass4321?'
    login_form = driver.find_elements(By.XPATH, f'//input[@class="form-control"]')
#     pdb.set_trace()
    
    if len(login_form) > 0:
        login_form_username = login_form[0]
        login_form_password = login_form[1]
        
        if need_un is True:
            login_form_username.send_keys(username)
            login_form_password.send_keys(password)
    
    UM.navigate(driver, xpath=f'//div/input[@type="submit"]', move=True, click=True)
    
nbc_login(driver)

## Search: Date range:

In [59]:
last_date = 'May 18 2022'

first_tab = driver.window_handles[0]
driver.switch_to.window(first_tab)

type_lst = ['Research Flash']
result = defaultdict(list)
flag = False

# Need to get page length:
last_page_button = driver.find_elements(By.XPATH, '//li[@class="paginate_button page-item "]')[-1]
total_pages = int(last_page_button.text)

for i in range(total_pages-1):
    content = UM.navigate(driver, xpath='//table[@id="searchDetails"]')
    trs = content.find_elements(By.XPATH, '//tr[@role="row"]')[1:]

    for tr in trs:
        tr_soup = BeautifulSoup(tr.get_attribute('innerHTML'), 'html.parser')
        tds = tr_soup.find_all('td')


        result['headline'].append(tds[0].find('a').text)
        result['url'].append(tds[0].find('a')['href'])
        result['ticker'].append(tds[1].find('a').text)
        result['analyst_pri'].append(tds[2].find('a').text)
        result['broker_label'].append(tds[3].text.strip('\n').strip())
        result['time'].append(tds[4].text.strip('\n').strip())
#     pdb.set_trace()
    UM.navigate(driver, xpath='//li[@class="paginate_button page-item next"]', move=True, click=True)

#     if flag == True: break    
#     driver.find_element_by_xpath('//*[@id="allReports_next"]/a').click()


In [72]:
test = DL.loadDB('test.csv', parse_dates=['time'])

19:01:51 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\Log/20220526 created.
19:01:51 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\test.csv


In [ ]:
df = result_df.copy()

In [213]:
for i in range(len(df)):

    driver.get(df.at[i,'url'].replace('pdf','html'))
    time.sleep(0.1)

    trs = driver.find_elements(By.XPATH, '//td/table/tbody')[1]
    soup = BeautifulSoup(trs.get_attribute('innerHTML'), 'html.parser')

    rating = soup.find_all('tbody')[0].find_all('td')[2].text.replace(u'\xa0', u' ').strip('\n ')
    rating = rating.split(': ')[1]
    df.at[i, 'rating_curr'] = rating
    
    tp = soup.find_all('tbody')[0].find_all('td')[3].text.replace(u'\xa0', u' ').strip('\n ')
    tp = tp.split('$')[1]
    df.at[i, 'tp_curr'] = tp
    
    summary = ' '.join([x.text for x in soup.find_all('p')])
    df.at[i, 'summary'] = summary
    
    uid = df.at[i,'url'].split('encrypt=')[1].split('&')[0]
    df.at[i,'report_id'] = uid
    
    time.sleep(0.002)

In [236]:
result_df

,headline,url,ticker,analyst_pri,broker_label,time,uid,rating_curr,tp_curr,summary,exch_location,exch_region,publish_date_and_time
0,"Portfolio Rationalized, La Guitarra Divested",https://nbf.bluematrix.com/sellside/EmailDocVi...,FR CN,Don DeMarco,Research Flash,May 25 2022 01:01PM,91d15aeb-cde1-4ad3-a281-9c6c417b291c,Sector Perform,18.00,Impact: Neutral FR announced a definitive agre...,Canada,Americas,May 25 2022 01:01PM
1,Triple Lynx Continues to Return Elevated Grades,https://nbf.bluematrix.com/sellside/EmailDocVi...,OSK CN,Don DeMarco,Research Flash,May 25 2022 10:13AM,f61cd17a-f581-47f7-b1a2-ca2195804dd9,Outperform,5.50,Impact: Neutral OSK released assays from its o...,Canada,Americas,May 25 2022 10:13AM
2,Rangefront Assay Update; Modest Improvement in...,https://nbf.bluematrix.com/sellside/EmailDocVi...,LGD CN,Rabi Nizami,Research Flash,May 25 2022 09:55AM,29aa0ef9-fce5-42a1-9d9e-2f5294e9f9a0,Outperform,1.80,"Impact: Neutral This morning, Liberty Gold pro...",Canada,Americas,May 25 2022 09:55AM
3,Q2/22 First Look – PCLs and trading revenues d...,https://nbf.bluematrix.com/sellside/EmailDocVi...,BMO CN,Gabriel Dechaine,Research Flash,May 25 2022 07:30AM,859d8afd-964e-4dfb-90b6-51308559eb20,Sector Perform,151.00,Event: BMO reported Q2/22 adjusted EPS of $3.2...,Canada,Americas,May 25 2022 07:30AM
4,Q2/22 First Look – Strong Canadian & Internati...,https://nbf.bluematrix.com/sellside/EmailDocVi...,BNS CN,Gabriel Dechaine,Research Flash,May 25 2022 07:05AM,03f02427-da32-451c-91fd-133a4664eb4f,Sector Perform,90.00,Event: BNS reported Q2/22 core cash EPS of $2....,Canada,Americas,May 25 2022 07:05AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Sandstorm Reports Q1/22 Sales and Revenue Ahea...,https://nbf.bluematrix.com/sellside/EmailDocVi...,SSL CN,Shane Nagle,Research Flash,Apr 04 2022 09:31AM,f4793833-8823-47bd-b481-fc84d4323b7d,Outperform,10.00,"Impact: Positive This morning, Sandstorm Gold ...",Canada,Americas,Apr 04 2022 09:31AM
246,Exploration Update,https://nbf.bluematrix.com/sellside/EmailDocVi...,YRI CN,Mike Parkin,Research Flash,Apr 04 2022 09:21AM,769d018b-362c-4805-bbec-42980669c455,Outperform,8.75,"This morning, Yamana reported an update on its...",Canada,Americas,Apr 04 2022 09:21AM
247,Copper Mountain Announces Commissioning of Tro...,https://nbf.bluematrix.com/sellside/EmailDocVi...,CMMC CN,Shane Nagle,Research Flash,Apr 04 2022 07:45AM,9b8c4155-46b9-4094-ad2f-dab814e9582d,Outperform,4.75,"Impact: Positive This morning, Copper Mountain...",Canada,Americas,Apr 04 2022 07:45AM
248,"Debt Term Sheet Closed, Supports Liquidity",https://nbf.bluematrix.com/sellside/EmailDocVi...,MOZ CN,Don DeMarco,Research Flash,Apr 01 2022 08:43AM,d4cde9bd-3cc8-4b85-b10e-1438eb716f60,Outperform,4.50,"Impact: Neutral After market close, MOZ report...",Canada,Americas,Apr 01 2022 08:43AM


### Manually change

In [100]:
# df.at[7, 'rating_prev'] = 'Underperform 4'
# df.at[8, 'rating_prev'] = 'Outperform 2'

In [237]:
# df.replace(column={'time': 'publish_date_and_time'}, inplace=True)
df = result_df.copy()
df['publish_date_and_time'] = df['time']
df[['ticker', 'rating_curr', 'tp_curr', 'time']]

,ticker,rating_curr,tp_curr,time
0,FR CN,Sector Perform,18.00,May 25 2022 01:01PM
1,OSK CN,Outperform,5.50,May 25 2022 10:13AM
2,LGD CN,Outperform,1.80,May 25 2022 09:55AM
3,BMO CN,Sector Perform,151.00,May 25 2022 07:30AM
4,BNS CN,Sector Perform,90.00,May 25 2022 07:05AM
...,...,...,...,...
245,SSL CN,Outperform,10.00,Apr 04 2022 09:31AM
246,YRI CN,Outperform,8.75,Apr 04 2022 09:21AM
247,CMMC CN,Outperform,4.75,Apr 04 2022 07:45AM
248,MOZ CN,Outperform,4.50,Apr 01 2022 08:43AM


In [245]:
df['headline_senti'] = None
df['summary_senti'] = None

df['headline_senti'], _ = update_sentiment(df['headline'])
df['summary_senti'], _ = update_sentiment(df['summary'])

2022-05-27 16:13:12.289416


C:\Users/JayYang/OneDrive - Alpha Sherpa Capital\finBERT\sentiments.py:40: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


Device: cuda
'update_sentiment'  13395.46 ms
2022-05-27 16:13:25.684855
Device: cuda
'update_sentiment'  14279.48 ms


In [321]:
sentiment_df = DL.loadDB('NBC sentiment.csv', parse_dates=['publish_date_and_time'])

16:48:47 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\NBC sentiment.csv


In [322]:
df['headline_senti'] = None
df['summary_senti'] = None

df[['tp_prev', 'tp_chg', 'tp_chg_pct', 'abs(tp_chg)']] = None
df[['rating_prev']] = None
df[['report_type']] = ''

if len(sentiment_df) == 0:
    new_df = df.copy()
else:
    sentiment_df = pd.concat([sentiment_df, df[sentiment_df.columns]], axis=0).drop_duplicates(['publish_date_and_time', 'uid'])
    new_df = sentiment_df[sentiment_df['headline_senti'].isna()].copy().reset_index(drop=True)
    sentiment_df = sentiment_df[~sentiment_df['headline_senti'].isna()].reset_index(drop=True)

In [323]:
if len(new_df) > 0:
    new_df['headline_senti'], _ = update_sentiment(new_df['headline'])
    new_df['summary_senti'], _ = update_sentiment(new_df['summary'])

2022-05-27 16:48:59.040515


C:\Users/JayYang/OneDrive - Alpha Sherpa Capital\finBERT\sentiments.py:40: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


Device: cuda
'update_sentiment'  13322.35 ms
2022-05-27 16:49:12.363733
Device: cuda
'update_sentiment'  14487.82 ms


In [324]:
DL.toDB(new_df, 'new_df.csv')
new_df = DL.loadDB('new_df.csv', parse_dates=['publish_date_and_time'])

16:49:26 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Database/NBC\new_df.csv


Index(['ticker', 'publish_date_and_time', 'uid', 'analyst_pri', 'headline',
       'summary', 'headline_senti', 'summary_senti', 'rating_curr',
       'rating_prev', 'tp_curr', 'tp_prev', 'tp_chg_pct', 'abs(tp_chg)',
       'report_type', 'tp_chg', 'ad-hoc', 'Earning's Review', 'Rating Change',
       'Target Price Change', 'Estimate Change', 'Initiation'],
      dtype='object')

In [325]:
sentiment_df = pd.concat([new_df[sentiment_df.columns], sentiment_df], axis=0).sort_values('publish_date_and_time', ascending=True).\
    drop_duplicates(['publish_date_and_time', 'uid'], keep='first').reset_index(drop=True)
sentiment_df['rating_prev'] = sentiment_df.groupby(['ticker'])['rating_curr'].apply(lambda x: x.shift())

# Clean target prices mannually with Artificial Intelligence

sentiment_df = sentiment_df.sort_values(['publish_date_and_time'], ascending=True).reset_index(drop=True)
sentiment_df['tp_curr'] = sentiment_df.groupby(['ticker'])['tp_curr'].apply(lambda x: x.fillna(method='ffill'))

sentiment_df['tp_curr'] = sentiment_df['tp_curr'].str.replace(',', '').str.strip('$ ')
# sentiment_df['tp_prev'] = sentiment_df['tp_prev'].fillna('').str.replace(',', '').str.strip('$ ')

for col in ['tp_curr', 'tp_chg_pct', 'abs(tp_chg)']:
    sentiment_df[col] = sentiment_df[col].astype(float)
sentiment_df['tp_prev'] = sentiment_df.groupby(['ticker'])['tp_curr'].apply(lambda x: x.shift())
sentiment_df['tp_chg'] = sentiment_df.groupby(['ticker'])['tp_curr'].apply(lambda x: x - x.shift())
sentiment_df['abs(tp_chg)'] = abs(sentiment_df['tp_chg'])
sentiment_df['tp_chg_pct'] = sentiment_df['tp_chg'] / sentiment_df['tp_curr']

sentiment_df['rating_curr'] = sentiment_df.groupby(['ticker'])['rating_curr'].apply(lambda x: x.fillna(method='ffill'))
sentiment_df['rating_prev'] = sentiment_df.groupby(['ticker'])['rating_curr'].apply(lambda x: x.shift())


# Define report types

er_mask = er_filter(sentiment_df['headline'], sentiment_df['summary'])
rc_mask = sentiment_df['rating_curr'] != sentiment_df['rating_prev']
# Backfill rating_prev with previous rating_curr

tp_mask = sentiment_df['tp_curr'] != sentiment_df['tp_prev']
io_mask = io_filter(sentiment_df['headline'], sentiment_df['summary'])
ec_mask = ec_filter(sentiment_df['headline'], sentiment_df['summary'])

sentiment_df['report_type'] = 'ad-hoc'
sentiment_df['report_type'] = np.where(er_mask, 'Earning\'s Review', sentiment_df['report_type'])
sentiment_df['report_type'] = np.where(ec_mask, 'Estimate Change', sentiment_df['report_type'])
sentiment_df['report_type'] = np.where(io_mask, 'Initiation', sentiment_df['report_type'])
sentiment_df['report_type'] = np.where(tp_mask, 'Target Price Change', sentiment_df['report_type'])
sentiment_df['report_type'] = np.where(rc_mask, 'Rating Change', sentiment_df['report_type'])

In [326]:
DL.toDB(sentiment_df, f'NBC sentiment.csv')

### Sentiments